In [20]:
import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write


HOSTAPI_FILTER = ''  # The name of the host API (driver) to use
NAME_FILTER = ''  # The name of the microphone to use
RATE = 44100  # Sample rate
DURATION = 5  # Duration to record
CHANNELS = 1  # Mono audio

# Find the host API matching the required filter
matching_hostapis = [idx for idx, h in enumerate(sd.query_hostapis()) if HOSTAPI_FILTER in h['name']]
if not matching_hostapis:
    raise ValueError(f'Hostapi matching name filter ({NAME_FILTER}) not found.')

idx_h = matching_hostapis[0]

matching_devices = [device for device in sd.query_devices()
                    if device['max_input_channels'] > 0
                    and NAME_FILTER in device.get('name')
                    and device['hostapi'] == idx_h]

if not matching_devices:
    raise ValueError(f'Device matching name filter ({NAME_FILTER}) not found.')

input_device = matching_devices[1]

# data recorded as 'float32' by default with +1.0 and -1.0 as the maximum and minimum values, respectively
data: np.ndarray = sd.rec(int(DURATION * RATE), samplerate=RATE, channels=CHANNELS, device=input_device["index"])

# Wait for the recording to finish
sd.wait()

# Save as WAV file
write("output.wav", RATE, data)

In [ ]:
mport pyaudio
import wave

CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 44100
RECORD_SECONDS = 5
WAVE_OUTPUT_FILENAME = "sound.wav"

p = pyaudio.PyAudio()

stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=CHUNK)

print("* recording")

frames = []

for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)

print("* done recording")

stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()